In [1]:
import requests

In [40]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
r = requests.get(url, allow_redirects=True)
open('adult.data', 'wb').write(r.content)

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test'
r = requests.get(url, allow_redirects=True)
open('adult.test', 'wb').write(r.content)


2003153

In [41]:
import pandas as pd
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'educational-num','marital-status', 'occupation', 'relationship', 'race', 'gender','capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income']

train = pd.read_csv('adult.data', sep=",\s", header=None, names = column_names, engine = 'python')
test = pd.read_csv('adult.test', sep=",\s", header=None, names = column_names, engine = 'python')

# test['income'].replace(regex=True,inplace=True,to_replace=r'\.',value=r'')

In [42]:
display(train.income.unique(), test.income.unique())

array(['<=50K', '>50K'], dtype=object)

array([None, '<=50K.', '>50K.'], dtype=object)

In [43]:
test.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,|1x3 Cross validator,None,NaN,None,NaN,None,None,None,None,None,NaN,NaN,NaN,None,None
1,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
2,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
3,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.
4,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K.


In [44]:
test.drop(axis=0, index=0, inplace=True)

In [45]:
test.income = [int(i == '>50K.') for i in test.income]
train.income = [int(i == '>50K') for i in train.income]
test_label = test.income
test =test.drop('income', axis=1)
train_label = train.income
train = train.drop('income', axis=1)

In [46]:
train.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [47]:
display(sum(train_label), sum(test_label))

7841

3846

In [48]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

integer_columns = ['age','fnlwgt','educational-num','capital-gain', 'capital-loss', 'hours-per-week']
categorical_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']


numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('one_hot', OneHotEncoder(handle_unknown='ignore'))])
    
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, integer_columns),
        ('cat', categorical_transformer, categorical_columns)])


In [49]:
from sklearn import svm
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', svm.SVC(kernel = 'rbf', C = 1, tol = 1e-3))])

clf.fit(train, train_label)
print("model score: %.3f" % clf.score(test, test_label))

model score: 0.860


In [50]:
from sklearn import svm
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', svm.SVC(kernel = 'linear'))])

clf.fit(train, train_label)
print("model score: %.3f" % clf.score(test, test_label))

model score: 0.853


In [51]:
from sklearn import svm
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', svm.SVC(kernel = 'poly'))])

clf.fit(train, train_label)
print("model score: %.3f" % clf.score(test, test_label))

model score: 0.857


In [52]:
from sklearn import svm
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', svm.SVC(kernel = 'sigmoid'))])

clf.fit(train, train_label)
print("model score: %.3f" % clf.score(test, test_label))

model score: 0.780


In [ ]:
https://archive.ics.uci.edu/ml/datasets/adult
https://github.com/saravrajavelu/Adult-Income-Analysis/blob/master/Adult_Income_Analysis.py

## One-Class SVM
Finde die kleinst mögliche Kugel (hyper sphere), die alle Datenpunkte umschliesst.
Angewendet auf neue Datenpunkte:
    - finde die Distanz zum Mittelpunkt: ist diese Grösser als der Radius der hyper sphere, dann ist der Datenpunkt nicht in der Klasse
    - finde die Distanz zur Begrenuzungsebene: pos Distanz -> in-class; neg Distanz -> out-class